In [49]:
import pandas as pd

# read in news articles
df_news = pd.read_csv("abcnews-date-text.csv")
df_news = df_news.sample(500)#too large to use all of them
df_news.rename(index=str, columns={"publish_date": "Emotion", "headline_text": "text"}, inplace=True)
df_news["Emotion"] = "False"

#read in emotional friends quotes
df_e = pd.read_json("friends_train.json")
df_e = df_e.append(pd.read_json("friends_test.json"))
df_e = df_e.append(pd.read_json("friends_dev.json"))
e_list = df_e[0].tolist()
emot = []
text = []
for line in e_list:
    emot.append(line["emotion"])
    text.append(line["utterance"])
df_emot = pd.DataFrame({"Emotion":emot, "text":text}, columns=["Emotion","text"])
df_emot = df_emot.loc[df_emot["Emotion"] != "neutral"]
df_emot = df_emot.loc[df_emot["Emotion"] != "non-neutral"]
df_emot["Emotion"] = "True"
df_emot = df_emot.sample(300)

#read in some of out tweets to test on
df_xl = pd.read_excel("emotions.xlsx")
df_xl["Emotion"].fillna(0,inplace=True)
df_xl["Emotion"] = df_xl["Emotion"].map({1:"True", 0:"False"})
d = {"Emotion":df_xl["Emotion"], "text":df_xl["Tweet"]}
df_coded = pd.DataFrame(data=d)

#create a shuffled dataframe of emotion and news tweets
df_train = pd.concat([df_news,df_emot,df_coded])
df_train = df_train.sample(frac=1).reset_index(drop=True)

#uncomment this when applying to my training set
#df_f = df_train.loc[df_train["Emotion"]=="False"]
#df_t = df_train.loc[df_train["Emotion"]=="True"]
#df_train = pd.concat([df_f.sample(27),df_t])
print(df_train["Emotion"].value_counts())

False    1212
True      575
Name: Emotion, dtype: int64


In [50]:
from pd_doc2vec import doc2vec
#use pd doc2vec class on training tweets, get cross val accuaracy
x = doc2vec(df_train, "text",["Emotion"], build=True)
print(x.score(verbose=True))

  0%|          | 0/90 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 90/90 [00:00<00:00, 1197.85it/s]
/home/nwest13/Curent/Syd/pd_doc2vec.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])
100%|██████████| 90/90 [00:00<00:00, 480.62it/s]


Accuracy Score:  0.9133333333333334
Label Score:  [0.9380091909596988, 0.8487676270657316]
[[0.9380091909596988, 0.8487676270657316], 0.9666666666666667]


In [46]:
#read in some of out tweets to test on
df_test = pd.read_excel("emotion_test.xlsx")
df_test = df_test.loc[df_test["Relevant"] == 1]
df_test["Emotion?"].fillna(0,inplace=True)
df_test["Emotion?"] = df_test["Emotion?"].map({1:"True", 0:"False"})
print(df_test.head())

   _Order           Source   categories created_at                       _id  \
1       2  Statuses_Irma_A  ['utility']  9/15/2017  5c11a746ec4ee522100a5b3b   
2       3  Statuses_Irma_A      ['gov']   9/8/2017  5c0c5cf2ec4ee5221008263d   
3       4  Statuses_Irma_A          NaN  5/11/2017  5c598cc9ec4ee5929fd1e356   
5       6  Statuses_Irma_C          NaN  6/20/2017  5c40997eec4ee5089818025a   
6       7  Statuses_Irma_A          NaN  9/16/2017  5c598ccbec4ee5929fd33dd2   

                                                text  emotion_ml  opinion_ml  \
1  I am happy to speak with someone at @dukeenerg...           0           1   
2  Consider it done! Great work! #9PMRoutine http...           0           1   
3    @WPLGLocal10 Waste of time and taxpayers mon ey           0           1   
5  Word to the wise David, I would appreciate you...           0           1   
6  @kylegravlin @WESH How can I watch it when the...           0           1   

   Relevant  Opinion? Emotion? Which e

In [47]:
#predict on hand coded tweets
predictions = (x.predict(df_test["text"])).tolist()
correct = df_test["Emotion?"].tolist()

  0%|          | 0/123 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 123/123 [00:00<00:00, 539.37it/s]


In [48]:
text = df_test["text"].tolist()

#get some accuracy
tp = 0
tn = 0
fp = 0
fn =0
for i in range(len(correct)):
    if(predictions[i] == "True"):
        if(correct[i] == "True"):
            tp = tp + 1
        else:
            fp = fp + 1
    else:
        if(correct[i] == "False"):
            tn = tn + 1
        else:
            fn = fn + 1
            #print(text[i])

precision = tp/(tp+fp)
recall = tp/(tp+fn)
accuracy = (tp+tn)/(tp+tn+fp+fn)
f1 = precision*recall/(precision+recall)
print("Total cases: "+str(len(correct)))
print("True Positives: "+str(tp))
print("False Positive: "+str(fp))
print("True Negatives: "+str(tn))
print("False Negatives: "+str(fn))
print("Accuracy: "+str(accuracy))
print("Precision: "+str(precision))
print("Recall: "+str(recall))
print("F1 Score: "+str(f1))

print(df_test["Emotion?"].value_counts())

Total cases: 123
True Positives: 111
False Positive: 12
True Negatives: 0
False Negatives: 0
Accuracy: 0.9024390243902439
Precision: 0.9024390243902439
Recall: 1.0
F1 Score: 0.4743589743589744
True     111
False     10
Name: Emotion?, dtype: int64
